# Ground up solution

First of all, this solution is built on the idea to practise the complete workflow of machine learning prediction. Even though auto ML and blending other people's results are frequently used and legit in Kaggle, I think building a ground up solution is beneficial for gaining really solid understanding of the ML techniques.

Since my computer is not powerful enough to handle the size of data here (100K times 50) with ease, I will strive for simple and efficient way to build the classification model on Kaggle hosting server. Again, the modelling process here is for learning the ML process rather than doing the fancy stuff or building specific solutions that are not ready for generalisation to other problems.

Based on the background above, you will see a solution in favour of simple ML workflow and low computation cost, ready to be deployed for different problems. The presentation may be raw, but I will keep it to show how the result is improved gradually.

Workflow:
1. Data Exploration
2. Data Preprocessing
3. Feature Engineering
4. Feature Selection
5. Model Validation And Selection
6. Hyperparameter Tuning


# Preparation

## Imports

In [ ]:
# Essentials
import numpy as np
import pandas as pd
import datetime
import random

# Plots
import seaborn as sns
import matplotlib.pyplot as plt

# Models
from sklearn.ensemble import RandomForestClassifier, GradientBoostingRegressor, AdaBoostRegressor, BaggingRegressor, ExtraTreesClassifier, StackingClassifier
from sklearn.kernel_ridge import KernelRidge
from sklearn.linear_model import RidgeClassifier, RidgeCV
from sklearn.linear_model import ElasticNet, ElasticNetCV, LogisticRegressionCV
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from mlxtend.classifier import StackingCVClassifier
import lightgbm as lgb
from lightgbm import LGBMClassifier
from xgboost import XGBClassifier
from catboost import Pool, CatBoostClassifier

# Stats
from scipy.stats import skew, norm
from scipy.special import boxcox1p
from scipy.stats import boxcox_normmax

# Misc
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV
from sklearn.model_selection import StratifiedKFold, cross_val_score, validation_curve
from sklearn.metrics import log_loss, confusion_matrix
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import LabelEncoder
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import scale
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import RobustScaler
from sklearn.decomposition import PCA
from sklearn.feature_selection import mutual_info_classif

pd.set_option('display.max_columns', None)

# Ignore useless warnings
import warnings
warnings.filterwarnings(action="ignore")
pd.options.display.max_seq_items = 8000
pd.options.display.max_rows = 8000

import os
os.listdir("../input/")

## Read data

In [ ]:
# Read in the dataset as a dataframe
train = pd.read_csv("../input/tabular-playground-series-may-2021/train.csv")
test = pd.read_csv("../input/tabular-playground-series-may-2021/test.csv")
submission = pd.read_csv("../input/tabular-playground-series-may-2021/sample_submission.csv")

#train.info()
#test.info()
#submission.info()

## Split datasets

In [ ]:
# Split features and labels
train_labels = train['target'].reset_index(drop=True)
train_features = train.drop(['id','target'], axis=1)
test_features = test.drop(['id'], axis=1)
train_labels.head()

# Data Exploration

## Target distribution

As observed, 57% of the target in the training set is of "Class 2", which is moderately imbalanced.

In [ ]:

sns.set_style("white")
sns.set_color_codes(palette='deep')
f, ax = plt.subplots(figsize=(8, 7))
#Check the new distribution 
sns.histplot(train['target'].sort_values(), color="b");
ax.xaxis.grid(False)
ax.set(ylabel="Frequency")
ax.set(xlabel="Target")
ax.set(title="Target distribution")
sns.despine(trim=True, left=True)
plt.show()


In [ ]:
train['target'].value_counts().sort_values(ascending=False)/sum(train['target'].value_counts())

## Features EDA

No specific pattern is observed in this case.

In [ ]:

# visualising some more outliers in the data values
fig, axs = plt.subplots(ncols=2, nrows=1, figsize=(12, 120))
plt.subplots_adjust(right=2)
plt.subplots_adjust(top=2)
sns.color_palette("husl", 8)
for i, feature in enumerate(list(train_features), 1):
    plt.subplot(len(list(train_features)), 3, i)
    sns.boxplot(x=feature, y=train_labels, hue=train_labels, palette='Blues', data=train_features)
        
    plt.xlabel('{}'.format(feature), size=15,labelpad=12.5)
    plt.ylabel('Target', size=15, labelpad=12.5)
    
    for j in range(2):
        plt.tick_params(axis='x', labelsize=12)
        plt.tick_params(axis='y', labelsize=12)
    
    plt.legend(loc='best', prop={'size': 10})
        
plt.show()


## Correlation

Filter by RF feature importance first when the number of features is too large.

The 50 features show no significant correlation with each other.

In [ ]:

# Random Forest Classifier
rf = RandomForestClassifier(n_estimators=100, random_state=42)

rf_model = rf.fit(train_features, train_labels)
#rf_pred = rf_model.predict_proba(test_features)

forest_importances = pd.Series(rf.feature_importances_, index=train_features.columns)
top_feat = forest_importances.sort_values(ascending = False).head(20)
top_feat

train_features[top_feat.index]


In [ ]:

#corr = train_features[top_feat.index].corr()
#corr
corr = train.corr()
plt.subplots(figsize=(15,12))
sns.heatmap(corr, vmax=0.9, cmap="Blues", square=True)


### Further exploration for high correlation to target

The most important features by RF is feature_38, but visually its standalone correlation with the target is insignificant.

In [ ]:

data = pd.concat([train['feature_38'], train['target']], axis=1)
f, ax = plt.subplots(figsize=(8, 6))
fig = sns.boxplot(x=train['feature_38'], y="target", data=data)
#fig.axis(ymin=0, ymax=800000);


# Data Preprocessing

No outliers or missing values observed from EDA.

## Recombine datasets

No treatment is needed in this case.

# Feature Engineering

Observed that the training set has large amount of zero values in the original 50 features, I have tried to add 50 binary features depending on whether each of the original 50 features is zero or not. The trial does not provide significant improvement in preliminary RF model, so this idea is not adopted. There maybe information loss in grouping all nonzero values together, which leads to worse performance.

In [ ]:
'''
# feature of zero or nonzero values

def zeroornot(res, ls):
    m = res.shape[1]
    for l in ls:
        res = res.assign(newcol=pd.Series(res[l] == 0).astype(int)) 
        res.columns.values[m] = l + '_zero'
        m += 1
    return res

train_features_zero = zeroornot(train_features, train_features.columns.tolist())
test_features_zero = zeroornot(test_features, test_features.columns.tolist())
'''

In [ ]:
#train_features_zero = train_features_zero.drop(train_features_zero.iloc[:,0:50], axis=1)
#test_features_zero = test_features_zero.drop(test_features_zero.iloc[:,0:50], axis=1)

## PCA

Since there are 50 features, the dimension reduction technique may help. I have tried PCA, but the result is not satisfactory. This is intuitive given the low features correlation shown in EDA, and the almost identical contributions from all the principal components.

In [ ]:
'''
X=train_features
# Standardize
X_scaled = (X - X.mean(axis=0)) / X.std(axis=0)

# Create principal components
pca = PCA()
X_pca = pca.fit_transform(X_scaled)

# Convert to dataframe
component_names = [f"PC{i+1}" for i in range(X_pca.shape[1])]
X_pca = pd.DataFrame(X_pca, columns=component_names)

X_pca.head()
'''

In [ ]:
'''
loadings = pd.DataFrame(
    pca.components_.T,  # transpose the matrix of loadings
    columns=component_names,  # so the columns are the principal components
    index=X.columns,  # and the rows are the original features
)
loadings
'''

In [ ]:
'''
def plot_variance(pca, width=8, dpi=100):
    # Create figure
    fig, axs = plt.subplots(1, 2)
    n = pca.n_components_
    grid = np.arange(1, n + 1)
    # Explained variance
    evr = pca.explained_variance_ratio_
    axs[0].bar(grid, evr)
    axs[0].set(
        xlabel="Component", title="% Explained Variance", ylim=(0.0, 0.05)
    )
    # Cumulative Variance
    cv = np.cumsum(evr)
    axs[1].plot(np.r_[0, grid], np.r_[0, cv], "o-")
    axs[1].set(
        xlabel="Component", title="% Cumulative Variance", ylim=(0.0, 1.0)
    )
    # Set up figure
    fig.set(figwidth=8, dpi=100)
    return axs

# Look at explained variance
plot_variance(pca);
'''

In [ ]:
'''
def make_mi_scores(X, y, discrete_features):
    mi_scores = mutual_info_classif(X, y, discrete_features=discrete_features)
    mi_scores = pd.Series(mi_scores, name="MI Scores", index=X.columns)
    mi_scores = mi_scores.sort_values(ascending=False)
    return mi_scores

mi_scores = make_mi_scores(X_pca, train_labels, discrete_features=False)
mi_scores
'''

In [ ]:
'''
miindex = mi_scores.index[mi_scores.values>0]
miload = loadings[miindex]
'''

In [ ]:
'''
train_features_pca = pd.DataFrame(data = np.matmul(train_features,miload))
train_features_pca.columns=miindex
'''

In [ ]:
'''
test_features_pca = pd.DataFrame(data = np.matmul(test_features,miload))
test_features_pca.columns=miindex
'''

## Recreate training and test sets

No treatment is needed in this case.

# Feature Selection

# Model Validation and Selection

## Set up CV

In [ ]:
# Set up cross validation folds
kf = StratifiedKFold(n_splits=5, random_state=42, shuffle=True)

# Define error metrics
def loss(y, y_pred):
    return np.sqrt(log_loss(y, y_pred))

def cv_loss(model, X = train_features):
    loss = np.sqrt(-cross_val_score(model, X, train_labels, scoring="neg_log_loss", cv=kf, n_jobs=-1))
    return (loss)

## Base Models

### XGBoost

In [ ]:

# XGBoost Classifier
xgb =  XGBClassifier(learning_rate = 0.1,
                        colsample_bytree = 0.5,
                        max_depth = 10,
                        min_child_weight=5,
                       gamma=0.001,
                       subsample=0.9,
                       objective='multi:softprob',
                       nthread=-1,
                       scale_pos_weight=1,
                       reg_alpha=0.00006,
                       random_state=42)

#xgb_model = xgb.fit(train_features, train_labels)
#xgb_pred = xgb_model.predict_proba(test_features)


In [ ]:

# XGBoost Classifier2
xgb2 = XGBClassifier(n_estimators=110,
                        learning_rate = 0.5,
                        colsample_bytree = 0.13,
                       max_depth = 2,
                        min_child_weight=5,
                       gamma=0.001,
                       subsample=0.7,
                       objective='multi:softprob',
                       nthread=-1,
                       scale_pos_weight=1,
                       reg_alpha=0.00006,
                       random_state=42)

#xgb2_model = xgb2.fit(train_features, train_labels)
#xgb2_pred = xgb2_model.predict_proba(test_features)

In [ ]:

# XGBoost Classifier3
xgb3 =XGBClassifier(n_estimators=250,
                        learning_rate = 0.06,
                        colsample_bytree = 0.4,
                       max_depth = 6,
                        min_child_weight=5,
                       subsample=0.75,
                       objective='multi:softprob',
                       nthread=-1,
                       random_state=42)
                       

#xgb3_model = xgb3.fit(train_features, train_labels)
#xgb3_pred = xgb3_model.predict_proba(test_features)

In [ ]:
# XGBoost Classifier4
xgb4 = XGBClassifier(n_estimators=130, 
                        learning_rate = 0.5,
                        colsample_bytree = 0.13,
                        max_depth = 2,
                        min_child_weight=7, 
                       gamma=0.001,
                       subsample=0.7,
                       objective='multi:softprob',
                       nthread=-1,
                       reg_alpha=0.00006,
                       random_state=42)
                       

#xgb4_model = xgb4.fit(train_features, train_labels)
#xgb4_pred = xgb4_model.predict_proba(test_features)

In [ ]:
# XGBoost Classifier5
xgb5 = XGBClassifier( n_estimators=200,
                        learning_rate = 0.5,
                       colsample_bytree = 0.1,
                       max_depth = 2,
                        min_child_weight=5,
                      gamma=0.005,
                       subsample=0.7,
                       objective='multi:softprob',
                       nthread=-1,
                       scale_pos_weight=1,
                       reg_alpha=5,
                         reg_lambda = 0.3,
                         eval_metric = 'mlogloss',
                       random_state=42)
                       

#xgb5_model = xgb5.fit(train_features, train_labels)
#xgb5_pred = xgb5_model.predict_proba(test_features)

### Random Forest

In [ ]:

# Random Forest Classifier
rf = RandomForestClassifier(min_samples_split = 5,
                            min_samples_leaf = 5,
                            max_depth = None,
                            bootstrap = True,
                            n_jobs=-1,
                            criterion = "entropy",
                            n_estimators=500,
                            max_features = 12,
                            random_state = 42)
'''
rf_model = rf.fit(train_features, train_labels)
rf_pred = rf_model.predict_proba(test_features)
'''

### Light GBM

In [ ]:

# Light Gradient Boosting Regressor
lgb =  LGBMClassifier(objective='multiclass', 
                       num_leaves=6,
                    max_depth=6,
                       learning_rate=0.1, 
                       n_estimators=220,
                       max_bin=200, 
                       bagging_fraction=0.8,
                       bagging_freq=4, 
                       bagging_seed=8,
                       feature_fraction=0.7,
                       feature_fraction_seed=8,
                            verbose=-1,
                       random_state=17,
                   n_jobs=-1)

lgb_model = lgb.fit(train_features, train_labels)
lgb_pred = lgb_model.predict_proba(test_features)


In [ ]:

# Light Gradient Boosting Regressor
lgb2 =  LGBMClassifier(objective='multiclass', 
                       num_leaves=2,
                    max_depth=1,
                       learning_rate=0.5, 
                       n_estimators=550,
                      max_bin=25, 
                       bagging_fraction=0.6,
                       bagging_freq=8, 
                       bagging_seed=8,
                      feature_fraction=0.3,
                       feature_fraction_seed=8,
                            verbose=-1,
                       random_state=17,
                   n_jobs=-1)

#lgb2_model = lgb2.fit(train_features, train_labels)
#lgb2_pred = lgb2_model.predict_proba(test_features)

In [ ]:

# Light Gradient Boosting Regressor
lgb3 = LGBMClassifier(objective='multiclass', 
                      min_data_in_leaf=7, 
                    max_depth=4,
                       learning_rate=0.066, 
                       n_estimators=200,
                       bagging_fraction=0.75,
                       bagging_freq=4, 
                       bagging_seed=8,
                       feature_fraction=0.3,
                       feature_fraction_seed=8,
                            verbose=-1,
                       random_state=17,
                   n_jobs=-1)

#lgb3_model = lgb3.fit(train_features, train_labels)
#lgb3_pred = lgb3_model.predict_proba(test_features)

In [ ]:

# Light Gradient Boosting Regressor
lgb4 = LGBMClassifier(objective='multiclass', 
                       num_leaves=6,
                    max_depth=6,
                       learning_rate=0.1, 
                       n_estimators=250, 
                       max_bin=200, 
                       bagging_fraction=0.8,
                       bagging_freq=4, 
                       bagging_seed=8,
                       feature_fraction=0.5,
                       feature_fraction_seed=8,
                            verbose=-1,
                       random_state=17,
                   n_jobs=-1)

#lgb4_model = lgb4.fit(train_features, train_labels)
#lgb4_pred = lgb4_model.predict_proba(test_features)

In [ ]:

# Light Gradient Boosting Regressor
lgb5 = LGBMClassifier(objective='multiclass', 
                       num_leaves=6,
                    min_data_in_leaf=19, 
                    max_depth=6,
                       learning_rate=0.1, 
                      n_estimators=310, 
                       max_bin=80, 
                      bagging_fraction=0.8,
                       bagging_freq=4, 
                       bagging_seed=8,
                       feature_fraction=0.1, 
                       feature_fraction_seed=8,
                            verbose=-1,
                       random_state=17,
                   n_jobs=-1)

#lgb5_model = lgb5.fit(train_features, train_labels)
#lgb5_pred = lgb5_model.predict_proba(test_features)

In [ ]:

# Light Gradient Boosting Regressor
lgb6 = LGBMClassifier(objective='multiclass', 
                       num_leaves=6,
                    min_data_in_leaf=19, 
                    max_depth=5,
                      learning_rate=0.1, 
                      n_estimators=310, 
                       max_bin=20, 
                      bagging_fraction=0.8,
                       bagging_freq=4, 
                       bagging_seed=8,
                      feature_fraction=0.1, 
                       feature_fraction_seed=8,
                           lambda_l2 = 0.003,
                           min_gain_to_split = 0.0001,
                           metric = 'multi_logloss',
                            verbose=-1,
                       random_state=17,
                   n_jobs=-1)

#lgb6_model = lgb6.fit(train_features, train_labels)
#lgb6_pred = lgb6_model.predict_proba(test_features)

In [ ]:

# Light Gradient Boosting Regressor
lgb7 = LGBMClassifier(objective='multiclass', 
                       num_leaves=6,
                    min_data_in_leaf=19, 
                    max_depth=5,
                      learning_rate=0.1, 
                     n_estimators=500, 
                       max_bin=20, 
                      bagging_fraction=0.8,
                       bagging_freq=4, 
                       bagging_seed=8,
                      feature_fraction=0.1, 
                       feature_fraction_seed=8,
                          lambda_l1 = 9,
                           lambda_l2 = 0.003,
                           min_gain_to_split = 0.0001,
                           metric = 'multi_logloss',
                            verbose=-1,
                       random_state=17,
                   n_jobs=-1)

#lgb7_model = lgb7.fit(train_features, train_labels)
#lgb7_pred = lgb7_model.predict_proba(test_features)

### Extra Trees

In [ ]:

# Extra Trees Classifier
ext =ExtraTreesClassifier(  min_samples_split = 5,
                            min_samples_leaf = 5,
                            max_depth = 15,
                            bootstrap = True,
                            n_jobs=-1,
                            n_estimators=10,
                            max_features = 20,
                            random_state = 42,
                            criterion = 'entropy')
'''
ext_model = ext.fit(train_features, train_labels)
ext_pred = ext_model.predict_proba(test_features)
'''

### CatBoost

In [ ]:

#CatBoost
cat_features = train_features.columns.values.tolist()

train_dataset = Pool(data=train_features,
                     label=train_labels,
                     cat_features=cat_features)

eval_dataset = Pool(data=test_features,
                    cat_features=cat_features)

# Initialize CatBoostClassifier
cat =  CatBoostClassifier(n_estimators=500,
                           learning_rate=0.3,
                           max_depth=2,
                           loss_function='MultiClass',
                          random_state=17,
                          thread_count=-1,
                            colsample_bylevel=0.5,
                            min_data_in_leaf=5,
                            verbose=0)
# Fit model
#cat.fit(train_dataset)
# Get predicted probabilities for each class
#cat_pred = cat.predict_proba(eval_dataset)


In [ ]:

#CatBoost2


# Initialize CatBoostClassifier
cat2 = CatBoostClassifier(n_estimators=550,
                           learning_rate=0.5,
                           max_depth=1,
                           loss_function='MultiClass',
                          random_state=17,
                          thread_count=-1,
                            colsample_bylevel=0.4,
                            min_data_in_leaf=5,
                            verbose=0)
# Fit model
#cat2.fit(train_dataset)
# Get predicted probabilities for each class
#cat2_pred = cat2.predict_proba(eval_dataset)

In [ ]:

#CatBoost3


# Initialize CatBoostClassifier
cat3 = CatBoostClassifier(n_estimators=250,
                           learning_rate=0.1,
                           max_depth=6,
                           loss_function='MultiClass',
                          random_state=17,
                          thread_count=-1,
                            colsample_bylevel=0.4,
                            min_data_in_leaf=3,
                            verbose=0)

# Fit model
#cat3.fit(train_dataset)
# Get predicted probabilities for each class
#cat3_pred = cat3.predict_proba(eval_dataset)

In [ ]:
#CatBoost4

# Initialize CatBoostClassifier

cat4 = CatBoostClassifier(n_estimators=600,
                           learning_rate=0.6,
                           max_depth=1,
                           loss_function='MultiClass',
                          random_state=17,
                          thread_count=-1,
                            colsample_bylevel=0.4,
                            min_data_in_leaf=5,
                            verbose=0)


# Fit model
#cat4.fit(train_dataset)
# Get predicted probabilities for each class
#cat4_pred = cat4.predict_proba(eval_dataset)

In [ ]:
#CatBoost5

# Initialize CatBoostClassifier

cat5 = CatBoostClassifier(n_estimators=600,
                           learning_rate=0.6,
                           max_depth=1,
                           loss_function='MultiClass',
                          random_state=17,
                          thread_count=-1,
                            colsample_bylevel=0.4,
                            min_data_in_leaf=5,
                               bagging_temperature = 0.2,
                             sampling_frequency = 'PerTreeLevel',
                               reg_lambda = 3,
                            verbose=0)

# Fit model
#cat5.fit(train_dataset)
# Get predicted probabilities for each class
#cat5_pred = cat5.predict_proba(eval_dataset)

## Stacking

Only one level of meta model is trained on the base model predictions and the original features. The type of meta models would still be chosen from the available types in the base models. 

In [ ]:
%%time
# Stack up all the models above, optimized using lgb
stack_gen = StackingCVClassifier(classifiers = (rf, lgb, xgb, ext),
                                meta_classifier = lgb,
                                 use_probas= True,
                                use_features_in_secondary=True, 
                                 verbose=2,
                                 n_jobs=-1,
                                random_state=17)

#stack_gen_model = stack_gen.fit(np.array(train_features), np.array(train_labels))
#stack_pred = stack_gen_model.predict_proba(np.array(test_features))


In [ ]:

%%time
# Stack2
# Stack up all the models above, optimized using cat
stack_gen2 = StackingCVClassifier(classifiers = (xgb2, lgb, cat),
                                meta_classifier = cat,
                                 use_probas= True,
                                use_features_in_secondary=True, 
                                 verbose=2,
                                 n_jobs=-1,
                                random_state=17)

#stack_gen2_model = stack_gen2.fit(np.array(train_features), np.array(train_labels))
#stack2_pred = stack_gen2_model.predict_proba(np.array(test_features))


In [ ]:
%%time
# Stack4
# Stack up all the models above, optimized using cat2
stack_gen4 = StackingCVClassifier(classifiers = (xgb2,lgb, cat2),
                                meta_classifier = cat,
                                 use_probas= True,
                                use_features_in_secondary=True, 
                                 verbose=2,
                                 n_jobs=-1,
                                random_state=17
                                )

#stack_gen4_model = stack_gen4.fit(np.array(train_features), np.array(train_labels))
#stack4_pred = stack_gen4_model.predict_proba(np.array(test_features))

In [ ]:
%%time
# Stack5
# Stack up all the models above, optimized using cats5

cats5 = CatBoostClassifier(n_estimators=450, 
                           learning_rate=0.1, 
                           max_depth=1,
                           loss_function='MultiClass',
                          random_state=17,
                          thread_count=-1,
                            colsample_bylevel=0.5,
                            min_data_in_leaf=5,
                            verbose=0)
                            

stack_gen5 = StackingCVClassifier(classifiers = (xgb2,lgb, cat2),
                                meta_classifier = cats5,
                                 use_probas= True,
                                use_features_in_secondary=True, 
                                 verbose=2,
                                 n_jobs=-1,
                                random_state=17,
                                   stratify = True)


#stack_gen5_model = stack_gen5.fit(np.array(train_features), np.array(train_labels))
#stack5_pred = stack_gen5_model.predict_proba(np.array(test_features))

In [ ]:
%%time
# Stack6
# Stack up all the models above, optimized using lgbs6

lgbs6 = LGBMClassifier(objective='multiclass', 
                       num_leaves=6,
                    max_depth=6,
                       learning_rate=0.03, 
                       n_estimators=160, 
                       max_bin=200, 
                       bagging_fraction=0.8,
                       bagging_freq=4, 
                       bagging_seed=8,
                       feature_fraction=0.7,
                       feature_fraction_seed=8,
                            verbose=-1,
                       random_state=17,
                   n_jobs=-1)
                            

stack_gen6 = StackingCVClassifier(classifiers = (lgb, xgb, cat),
                                meta_classifier = lgbs6,
                                 use_probas= True,
                                use_features_in_secondary=True, 
                                 verbose=2,
                                 n_jobs=-1,
                                random_state=17,
                                   stratify = True)


#stack_gen6_model = stack_gen6.fit(np.array(train_features), np.array(train_labels))
#stack6_pred = stack_gen6_model.predict_proba(np.array(test_features))

In [ ]:
%%time
# Stack7
# Stack up all the models above, optimized using lgbs7


lgbs7 = LGBMClassifier(objective='multiclass', 
                      min_data_in_leaf=7, 
                    max_depth=4,
                       learning_rate=0.05, 
                       n_estimators=100,
                       bagging_fraction=1,
                       bagging_freq=4, 
                       bagging_seed=8,
                       feature_fraction=0.3,
                       feature_fraction_seed=8,
                            verbose=-1,
                       random_state=17,
                   n_jobs=-1)

stack_gen7 =  StackingCVClassifier(classifiers = (lgb3, xgb3, cat),
                                meta_classifier = lgbs7,
                                 use_probas= True,
                                use_features_in_secondary=True, 
                                 verbose=2,
                                 n_jobs=-1,
                                random_state=17,
                                   stratify = True)


#stack_gen7_model = stack_gen7.fit(np.array(train_features), np.array(train_labels))
#stack7_pred = stack_gen7_model.predict_proba(np.array(test_features))

In [ ]:
%%time
# Stack8
# Stack up all the models above, optimized using lgbs8


lgbs8 = LGBMClassifier(objective='multiclass', 
                     min_data_in_leaf=3, 
                    max_depth=4,
                       learning_rate=0.05, 
                       n_estimators=100,
                       bagging_fraction=0.75,
                       bagging_freq=4, 
                       bagging_seed=8,
                       feature_fraction=0.3,
                       feature_fraction_seed=8,
                            verbose=-1,
                       random_state=17,
                   n_jobs=-1)

stack_gen8 = StackingCVClassifier(classifiers = (xgb2, lgb, cat2),
                                meta_classifier = lgbs8,
                                 use_probas= True,
                                use_features_in_secondary=True, 
                                 verbose=2,
                                 n_jobs=-1,
                                random_state=17,
                                   stratify = True)

#stack_gen8_model = stack_gen8.fit(np.array(train_features), np.array(train_labels))
#stack8_pred = stack_gen8_model.predict_proba(np.array(test_features))

In [ ]:
%%time
# Stack9
# Stack up all the models above, optimized using lgbs9


lgbs9 = LGBMClassifier(objective='multiclass', 
                       num_leaves=6,
                   min_data_in_leaf=23, 
                    max_depth=3,
                       learning_rate=0.05, 
                    n_estimators=270, 
                      max_bin=60, 
                    bagging_fraction=0.7, 
                       bagging_freq=4, 
                       bagging_seed=8,
                       feature_fraction=0.1, 
                       feature_fraction_seed=8,
                            verbose=-1,
                       random_state=17,
                   n_jobs=-1)

stack_gen9 = StackingCVClassifier(classifiers = (xgb4, lgb5, cat4),
                                meta_classifier = lgbs9,
                                 use_probas= True,
                                use_features_in_secondary=True, 
                                 verbose=2,
                                 n_jobs=-1,
                                random_state=17,
                                   stratify = True)

#stack_gen9_model = stack_gen9.fit(np.array(train_features), np.array(train_labels))
#stack9_pred = stack_gen9_model.predict_proba(np.array(test_features))

In [ ]:
%%time
# Stack10
# Stack up all the models above, optimized using lgbs10


lgbs10 = LGBMClassifier(objective='multiclass', 
                       num_leaves=4,
                  min_data_in_leaf=400, 
                    max_depth=1, 
                     learning_rate=0.05, 
                   n_estimators=270, 
                      max_bin=60, 
                    bagging_fraction=0.5, 
                       bagging_freq=4, 
                       bagging_seed=8,
                       feature_fraction=0.15, 
                       feature_fraction_seed=8,
                            verbose=-1,
                       random_state=17,
                   n_jobs=-1)

                            

stack_gen10 = StackingCVClassifier(classifiers = (xgb4, lgb5, cat4),
                                meta_classifier = lgbs10,
                                 use_probas= True,
                                use_features_in_secondary= False, 
                                 verbose=2,
                                 n_jobs=-1,
                                random_state=17,
                                   stratify = True)


#stack_gen10_model = stack_gen10.fit(np.array(train_features), np.array(train_labels))
#stack10_pred = stack_gen10_model.predict_proba(np.array(test_features))

In [ ]:
%%time
# Stack11
# Stack up all the models above, optimized using lgbs11


lgbs11 = LGBMClassifier(objective='multiclass', 
                       num_leaves=5,
                    min_data_in_leaf=19, 
                    max_depth=4,
                      learning_rate=0.1, 
                     n_estimators=180, 
                       max_bin=20, 
                      bagging_fraction=0.8,
                       bagging_freq=4, 
                       bagging_seed=8,
                      feature_fraction=0.1, 
                       feature_fraction_seed=8,
                          lambda_l1 = 7,
                           lambda_l2 = 0.1,
                           min_gain_to_split = 0.01,
                           metric = 'multi_logloss',
                            verbose=-1,
                       random_state=17,
                   n_jobs=-1)
                       
stack_gen11 = StackingCVClassifier(classifiers = (xgb5, lgb7, cat5),
                                meta_classifier = lgbs11,
                                 use_probas= True,
                                use_features_in_secondary= True, 
                                 verbose=2,
                                 n_jobs=-1,
                                random_state=17,
                                   stratify = True)


stack_gen11_model = stack_gen11.fit(np.array(train_features), np.array(train_labels))
stack11_pred = stack_gen11_model.predict_proba(np.array(test_features))

## CV result

In [ ]:

scores = {}

score = cv_loss(stack_gen11)
print("rf: {:.4f} ({:.4f})".format(score.mean(), score.std()))
scores['rf'] = (score.mean(), score.std())


Redefine CV function for PCA

In [ ]:
'''
def cvpca_loss(model, X = train_features_pca):
    loss = np.sqrt(-cross_val_score(model, X, train_labels, scoring="neg_log_loss", cv=kf, n_jobs=-1))
    return (loss)

scores = {}

score = cvpca_loss(cat2)
print("rf: {:.4f} ({:.4f})".format(score.mean(), score.std()))
scores['rf'] = (score.mean(), score.std())
'''

## Confusion Matrix

The result shows that the imbalanced target data has led to prediction highly in favour of "Class 2", which is worth noting for insights.

In [ ]:
lgb_pred = lgb_model.predict(np.array(train_features))

In [ ]:
unique, counts = np.unique(lgb_pred, return_counts=True)
np.asarray((unique, counts)).T

In [ ]:
np.around(confusion_matrix(train_labels,lgb_pred, normalize = 'true'),3)

# Hyperparameter Tuning

## Validation Curves

Validation curve is most frequently used in hyperparameters tuning for its lower computation cost. Past studies have also shown that its performance is not far from the grid search if used properly.

In [ ]:
'''
%%time
num_est = [100,200,300]

vc_model = CatBoostClassifier(n_estimators=600,
                           learning_rate=0.6,
                           max_depth=1,
                           loss_function='MultiClass',
                          random_state=17,
                          thread_count=-1,
                            colsample_bylevel=0.4,
                           # min_data_in_leaf=5,
                               bagging_temperature = 0.2,
                             sampling_frequency = 'PerTreeLevel',
                               reg_lambda = 3,
                            verbose=0)



# Calculate accuracy on training and test set using the
# parameter with 5-fold cross validation
train_score, test_score = validation_curve( vc_model,
                                X = train_features, y = train_labels, 
                                param_name = 'min_data_in_leaf', 
                                param_range = num_est, cv = kf, scoring="neg_log_loss", n_jobs=-1
                            )
 
# Calculating mean and standard deviation of training score
mean_train_score = -np.mean(train_score, axis = 1)
std_train_score = np.std(train_score, axis = 1)
 
# Calculating mean and standard deviation of testing score
mean_test_score = -np.mean(test_score, axis = 1)
std_test_score = np.std(test_score, axis = 1)
 
# Plot mean accuracy scores for training and testing scores
plt.plot(num_est, mean_train_score,
     label = "Training Score", color = 'b')
plt.plot(num_est, mean_test_score,
   label = "Cross Validation Score", color = 'g')
 
# Creating the plot
plt.title("Validation Curve")
plt.xlabel("param")
plt.ylabel("LogLoss")
plt.tight_layout()
plt.legend(loc = 'best')
plt.show()
'''

In [ ]:
#mean_test_score

## Random Hyperparameter Grid

The random grid is mostly used as a preliminary to narrow down the range of hyperparameters for the finer grid search.

In [ ]:
'''
# Number of trees in random forest
n_estimators = [int(x) for x in np.linspace(start = 10, stop = 500, num = 10)]
# Number of features to consider at every split
max_features = [int(x) for x in np.linspace(5, 15, num = 10)]
max_features.append("auto")
# Maximum number of levels in tree
max_depth = [int(x) for x in np.linspace(2, 10, num = 5)]
max_depth.append(None)
# Minimum number of samples required to split a node
min_samples_split = 5
# Minimum number of samples required at each leaf node
min_samples_leaf = 5
# Method of selecting samples for training each tree
bootstrap = True

#XGB
learning_rate = [0.01,0.1,0.3,0.5, 0.7,1]
colsample_bytree = [0.05,0.1, 0.3, 0.5,0.7,1]

#LGB
feature_fraction = [0.1, 0.3, 0.5, 0.7, 0.9, 1]

#CAT
colsample_bylevel =[0.1, 0.3, 0.5, 0.7, 0.9, 1]

# Create the random grid
random_grid = {'n_estimators': n_estimators,
               #'max_features': max_features,
               'max_depth': max_depth,
               #'min_samples_split': min_samples_split,
               #'min_samples_leaf': min_samples_leaf,
               #'bootstrap': bootstrap,
               'learning_rate': learning_rate,
               #'colsample_bytree': colsample_bytree
               #'feature_fraction':feature_fraction,
               'colsample_bylevel':colsample_bylevel
              }
'''

In [ ]:
'''
# Use the random grid to search for best hyperparameters
# First create the base model to tune
rf = CatBoostClassifier(#n_estimators=550,
                           #learning_rate=0.5,
                           #max_depth=1,
                           loss_function='MultiClass',
                          random_state=17,
                          thread_count=-1,
                            #colsample_bylevel=0.4,
                            min_data_in_leaf=5,
                            reg_lambda = 3,
                            subsample = 0.8,
                            bootstrap_type = 'Bernoulli'

                            )
# Random search of parameters, using 2 fold cross validation, 
# search across 30 different combinations, and use all available cores
rf_random = RandomizedSearchCV(estimator = rf, param_distributions = random_grid, n_iter = 33, cv = 3, verbose=2, random_state=42, n_jobs = -1)
# Fit the random search model
rf_random.fit(train_features, train_labels)
'''

In [ ]:
#rf_random.best_params_

## Grid Search

In [ ]:
'''
# Create the parameter grid based on the results of random search 
param_grid = {
    #'max_depth': [5,7],
    'n_estimators': [220,240,260],
    #'learning_rate':[0.1,0.3,0.5],
    #'feature_fraction':[0.1,0.3],
    #'num_leaves':[2,6,10],
    #'bagging_fraction':[0.4,0.6,0.8],
    #'bagging_freq':[6,8,10],
    #'max_bin':[10,15,25],
    #'colsample_bylevel':[0.05,0.1,0.3],
    'min_data_in_leaf':[3,5,7],
    'reg_lambda':[1,3,5],
    'subsample':[0.6,0.8,1]
    
}
# Create a based model
rf = XGBClassifier(n_estimators=100,
                        learning_rate = 0.5,
                        colsample_bytree = 0.4,
                       max_depth = 6,
                        min_child_weight=5,
                       gamma=0.001,
                       subsample=0.7,
                       objective='multi:softprob',
                       nthread=-1,
                       scale_pos_weight=1,
                       reg_alpha=0.00006,
                       random_state=42)
)

# Instantiate the grid search model
grid_search = GridSearchCV(estimator = rf, param_grid = param_grid, 
                          cv = 3, n_jobs = -1, verbose = 2)
'''

In [ ]:
'''
# Fit the grid search to the data
grid_search.fit(train_features, train_labels)
#best_grid = 
grid_search.best_estimator_
'''

In [ ]:
#grid_search.best_params_

## Tuning for meta-model

In [ ]:
#stack_gen.get_params().keys()

In [ ]:
'''
%%time
num_est = [0.08,0.1,0.12]

vc_model = LGBMClassifier(objective='multiclass', 
                       num_leaves=5,
                    min_data_in_leaf=19, 
                    max_depth=4,
                      learning_rate=0.1, 
                     n_estimators=180, 
                       max_bin=20, 
                      bagging_fraction=0.8,
                       bagging_freq=4, 
                       bagging_seed=8,
                      feature_fraction=0.1, 
                       feature_fraction_seed=8,
                          lambda_l1 = 7,
                           lambda_l2 = 0.1,
                           min_gain_to_split = 0.01,
                           metric = 'multi_logloss',
                            verbose=-1,
                       random_state=17,
                   n_jobs=-1)
                       
stack_gen_vc = StackingCVClassifier(classifiers = (xgb5, lgb7, cat5),
                                meta_classifier = vc_model,
                                 use_probas= True,
                                use_features_in_secondary= True, 
                                 verbose=2,
                                 n_jobs=-1,
                                random_state=17,
                                   stratify = True)


# Calculate accuracy on training and test set using the
# parameter with 3-fold cross validation
train_score, test_score = validation_curve( stack_gen_vc,
                                X = train_features, y = train_labels, 
                                param_name = 'meta_classifier__learning_rate', 
                                param_range = num_est, cv = kf, scoring="neg_log_loss", n_jobs=-1
                            )
 
# Calculating mean and standard deviation of training score
mean_train_score = -np.mean(train_score, axis = 1)
std_train_score = np.std(train_score, axis = 1)
 
# Calculating mean and standard deviation of testing score
mean_test_score = -np.mean(test_score, axis = 1)
std_test_score = np.std(test_score, axis = 1)
 
# Plot mean accuracy scores for training and testing scores
plt.plot(num_est, mean_train_score,
     label = "Training Score", color = 'b')
plt.plot(num_est, mean_test_score,
   label = "Cross Validation Score", color = 'g')
 
# Creating the plot
plt.title("Validation Curve")
plt.xlabel("param")
plt.ylabel("LogLoss")
plt.tight_layout()
plt.legend(loc = 'best')
plt.show()
'''

In [ ]:
#mean_test_score

# Submission

In [ ]:

# Read in sample_submission dataframe
submission[['Class_1', 'Class_2', 'Class_3', 'Class_4']] = stack11_pred
submission.head()


In [ ]:
submission.to_csv("submission_stack11.csv", index=False)

## Performance Log

In [ ]:
'''
Performance
# CV scores have taken square root by careless setting, but would not affect the comparison of results.

Basic rf (n=100)
CV 1.0633 (0.0022)
public 1.12521

Featured rf (n=100)
CV 1.0643 (0.0024)
public 1.12140

Tuned rf
CV 1.0506 (0.0005)
public 1.09957

Tuned rf with criterion = "entropy"
CV 1.0501 (0.0007)
public 1.09912

Basic xgb
public 1.09495

Tuned xgb
CV 1.0473 (0.0007)
public 1.08944

Tuned2 xgb
CV 1.0453 (0.0006)
public 1.08896

Tuned3 xgb
CV 1.0455 (0.0005)
public 1.08767

Tuned4 xgb
CV 1.0453 (0.0007)
public 1.08834

Tuned5 xgb
CV 1.0445 (0.0007)
public 1.08785

Basic lgb
CV 1.0470 (0.0006)
public 1.09036

Tuned lgb
CV 1.0454 (0.0007)
public 1.08871

Tuned2 lgb
CV 1.0459 (0.0007)
public 1.08916

Tuned3 lgb
CV 1.0455 (0.0007)
public 1.08872

Tuned4 lgb
CV 1.0452 (0.0007)
public 1.08789

Tuned5 lgb
CV 1.0447 (0.0006)
public 1.08785

Tuned6 lgb
CV 1.0447 (0.0006)
public 1.08790

Tuned7 lgb
CV 1.0441 (0.0008)
public 1.08723

Basic ext
CV 1.0643 (0.0011)
public 1.12962

Tuned ext
CV 1.0550 (0.0004)
public 1.10729

Tuned2 ext
CV 1.0536 (0.0005)
public 1.10495

Basic cat
CV 1.0516 (0.0009)
public 1.10367

Tuned cat
CV 1.0450 (0.0008)
public 1.09039

Tuned2 cat
CV 1.0449 (0.0007)
public 1.09092

Tuned3 cat
CV 1.0456 (0.0006)
public 1.09146

Tuned4 cat
CV 1.0448 (0.0009)
public 1.09089

Tuned5 cat
CV 1.0446 (0.0007)
public 1.09073

Stacking (rf, xgb, lgb, ext)>lgb
CV 1.0455 (0.0007)
public 1.08811

Stacking1 (xgb, lgb, cat)>lgb
CV 1.0450 (0.0008)
public 1.08605

Stacking2 (xgb2, lgb, cat)>cat
CV 1.0448 (0.0008)
public 1.08766

Stacking3 (xgb2, lgb, cat)>xgb2
CV 1.0458 (0.0008)
public 1.08729

Stacking4 (xgb2, lgb, cat2)>cat
CV 1.0447 (0.0007)
public 1.08679

Stacking5 (xgb2, lgb, cat2)>cats5
CV 1.0443 (0.0008)
public 1.08685

Stacking6 (xgb, lgb, cat)>lgbs6
CV 1.0443 (0.0007)
public 1.08604

Stacking7 (xgb3, lgb3, cat)>lgbs7
CV 1.0443 (0.0008)
public 1.08641

Stacking8 (xgb2, lgb, cat2)>lgbs8
CV 1.0444 (0.0007)
public 1.08592

Stacking9 (xgb4, lgb5, cat4)>lgbs9
CV 1.0439 (0.0008)
public 1.08613

Stacking10 (Stacking9 with no features input to meta model)
CV 1.0440 (0.0007)
public 1.08674

Stacking11 (xgb5, lgb7, cat5)>lgbs11
CV 1.0436 (0.0008)
public 1.08627


rf = RandomForestClassifier(min_samples_split = 5,
                            min_samples_leaf = 5,
                            max_depth = None,
                            bootstrap = True,
                            n_jobs=-1,
                            criterion = "entropy",
                            n_estimators=500,
                            max_features = 12,
                            random_state = 42)

xgb1 = XGBClassifier(learning_rate = 0.1,
                        colsample_bytree = 0.5,
                        max_depth = 10,
                        min_child_weight=5,
                       gamma=0.001,
                       subsample=0.9,
                       objective='multi:softprob',
                       nthread=-1,
                       scale_pos_weight=1,
                       reg_alpha=0.00006,
                       random_state=42)
                       
xgb2 = XGBClassifier(n_estimators=110,
                        learning_rate = 0.5,
                        colsample_bytree = 0.13,
                       max_depth = 2,
                        min_child_weight=5,
                       gamma=0.001,
                       subsample=0.7,
                       objective='multi:softprob',
                       nthread=-1,
                       scale_pos_weight=1,
                       reg_alpha=0.00006,
                       random_state=42)
                       
                       
xgb3 = XGBClassifier(n_estimators=250,
                        learning_rate = 0.06,
                        colsample_bytree = 0.4,
                       max_depth = 6,
                        min_child_weight=5,
                       subsample=0.75,
                       objective='multi:softprob',
                       nthread=-1,
                       random_state=42)

xgb4 = XGBClassifier(n_estimators=130, 
                        learning_rate = 0.5,
                        colsample_bytree = 0.13,
                        max_depth = 2,
                        min_child_weight=7, 
                       gamma=0.001,
                       subsample=0.7,
                       objective='multi:softprob',
                       nthread=-1,
                       reg_alpha=0.00006,
                       random_state=42)

xgb5 = XGBClassifier( n_estimators=200,
                        learning_rate = 0.5,
                       colsample_bytree = 0.1,
                       max_depth = 2,
                        min_child_weight=5,
                      gamma=0.005,
                       subsample=0.7,
                       objective='multi:softprob',
                       nthread=-1,
                       scale_pos_weight=1,
                       reg_alpha=5,
                         reg_lambda = 0.3,
                         eval_metric = 'mlogloss',
                       random_state=42)
                       
lgb = LGBMClassifier(objective='multiclass', 
                       num_leaves=6,
                    max_depth=6,
                       learning_rate=0.1, 
                       n_estimators=220,
                       max_bin=200, 
                       bagging_fraction=0.8,
                       bagging_freq=4, 
                       bagging_seed=8,
                       feature_fraction=0.7,
                       feature_fraction_seed=8,
                            verbose=-1,
                       random_state=17,
                   n_jobs=-1)
                   
lgb2 = LGBMClassifier(objective='multiclass', 
                       num_leaves=2,
                    max_depth=1,
                       learning_rate=0.5, 
                       n_estimators=550,
                      max_bin=25, 
                       bagging_fraction=0.6,
                       bagging_freq=8, 
                       bagging_seed=8,
                      feature_fraction=0.3,
                       feature_fraction_seed=8,
                            verbose=-1,
                       random_state=17,
                   n_jobs=-1)
                   
lgb3 =  LGBMClassifier(objective='multiclass', 
                      min_data_in_leaf=7, 
                    max_depth=4,
                       learning_rate=0.066, 
                       n_estimators=200,
                       bagging_fraction=0.75,
                       bagging_freq=4, 
                       bagging_seed=8,
                       feature_fraction=0.3,
                       feature_fraction_seed=8,
                            verbose=-1,
                       random_state=17,
                   n_jobs=-1)

lgb4 = LGBMClassifier(objective='multiclass', 
                       num_leaves=6,
                    max_depth=6,
                       learning_rate=0.1, 
                       n_estimators=250, 
                       max_bin=200, 
                       bagging_fraction=0.8,
                       bagging_freq=4, 
                       bagging_seed=8,
                       feature_fraction=0.5,
                       feature_fraction_seed=8,
                            verbose=-1,
                       random_state=17,
                   n_jobs=-1)

lgb5 = LGBMClassifier(objective='multiclass', 
                       num_leaves=6,
                    min_data_in_leaf=19, 
                    max_depth=6,
                       learning_rate=0.1, 
                      n_estimators=310, 
                       max_bin=80, 
                      bagging_fraction=0.8,
                       bagging_freq=4, 
                       bagging_seed=8,
                       feature_fraction=0.1, 
                       feature_fraction_seed=8,
                            verbose=-1,
                       random_state=17,
                   n_jobs=-1)

lgb6 = LGBMClassifier(objective='multiclass', 
                       num_leaves=6,
                    min_data_in_leaf=19, 
                    max_depth=5,
                      learning_rate=0.1, 
                      n_estimators=310, 
                       max_bin=20, 
                      bagging_fraction=0.8,
                       bagging_freq=4, 
                       bagging_seed=8,
                      feature_fraction=0.1, 
                       feature_fraction_seed=8,
                           lambda_l2 = 0.003,
                           min_gain_to_split = 0.0001,
                           metric = 'multi_logloss',
                            verbose=-1,
                       random_state=17,
                   n_jobs=-1)

lgb7 = LGBMClassifier(objective='multiclass', 
                       num_leaves=6,
                    min_data_in_leaf=19, 
                    max_depth=5,
                      learning_rate=0.1, 
                     n_estimators=500, 
                       max_bin=20, 
                      bagging_fraction=0.8,
                       bagging_freq=4, 
                       bagging_seed=8,
                      feature_fraction=0.1, 
                       feature_fraction_seed=8,
                          lambda_l1 = 9,
                           lambda_l2 = 0.003,
                           min_gain_to_split = 0.0001,
                           metric = 'multi_logloss',
                            verbose=-1,
                       random_state=17,
                   n_jobs=-1)

                   
ext = ExtraTreesClassifier(  min_samples_split = 5,
                            min_samples_leaf = 5,
                            max_depth = 15,
                            bootstrap = True,
                            n_jobs=-1,
                            n_estimators=10,
                            max_features = 20,
                            random_state = 42,
                            criterion = 'entropy')
                            

cat = CatBoostClassifier(n_estimators=500,
                           learning_rate=0.3,
                           max_depth=2,
                           loss_function='MultiClass',
                          random_state=17,
                          thread_count=-1,
                            colsample_bylevel=0.5,
                            min_data_in_leaf=5,
                            verbose=0)
                            
                            
cat2 = CatBoostClassifier(n_estimators=550,
                           learning_rate=0.5,
                           max_depth=1,
                           loss_function='MultiClass',
                          random_state=17,
                          thread_count=-1,
                            colsample_bylevel=0.4,
                            min_data_in_leaf=5,
                            verbose=0)
                            
                            
cat3 =   CatBoostClassifier(n_estimators=250,
                           learning_rate=0.1,
                           max_depth=6,
                           loss_function='MultiClass',
                          random_state=17,
                          thread_count=-1,
                            colsample_bylevel=0.4,
                            min_data_in_leaf=3,
                            verbose=0)

cat4 = CatBoostClassifier(n_estimators=600,
                           learning_rate=0.6,
                           max_depth=1,
                           loss_function='MultiClass',
                          random_state=17,
                          thread_count=-1,
                            colsample_bylevel=0.4,
                            min_data_in_leaf=5,
                            verbose=0)

cat5 = CatBoostClassifier(n_estimators=600,
                           learning_rate=0.6,
                           max_depth=1,
                           loss_function='MultiClass',
                          random_state=17,
                          thread_count=-1,
                            colsample_bylevel=0.4,
                            min_data_in_leaf=5,
                               bagging_temperature = 0.2,
                             sampling_frequency = 'PerTreeLevel',
                               reg_lambda = 3,
                            verbose=0)


cats5 = CatBoostClassifier(n_estimators=450, 
                           learning_rate=0.1, 
                           max_depth=1,
                           loss_function='MultiClass',
                          random_state=17,
                          thread_count=-1,
                            colsample_bylevel=0.5,
                            min_data_in_leaf=5,
                            verbose=0)
                            
                            
lgbs6 = LGBMClassifier(objective='multiclass', 
                       num_leaves=6,
                    max_depth=6,
                       learning_rate=0.03, 
                       n_estimators=160, 
                       max_bin=200, 
                       bagging_fraction=0.8,
                       bagging_freq=4, 
                       bagging_seed=8,
                       feature_fraction=0.7,
                       feature_fraction_seed=8,
                            verbose=-1,
                       random_state=17,
                   n_jobs=-1)
                   
                   
lgbs7 = LGBMClassifier(objective='multiclass', 
                      min_data_in_leaf=7, 
                    max_depth=4,
                       learning_rate=0.05, 
                       n_estimators=100,
                       bagging_fraction=1,
                       bagging_freq=4, 
                       bagging_seed=8,
                       feature_fraction=0.3,
                       feature_fraction_seed=8,
                            verbose=-1,
                       random_state=17,
                   n_jobs=-1)
                   
lgbs8 = LGBMClassifier(objective='multiclass', 
                     min_data_in_leaf=3, 
                    max_depth=4,
                       learning_rate=0.05, 
                       n_estimators=100,
                       bagging_fraction=0.75,
                       bagging_freq=4, 
                       bagging_seed=8,
                       feature_fraction=0.3,
                       feature_fraction_seed=8,
                            verbose=-1,
                       random_state=17,
                   n_jobs=-1)
                   
lgbs9 =  LGBMClassifier(objective='multiclass', 
                       num_leaves=6,
                   min_data_in_leaf=23, 
                    max_depth=3,
                       learning_rate=0.05, 
                    n_estimators=270, 
                      max_bin=60, 
                    bagging_fraction=0.7, 
                       bagging_freq=4, 
                       bagging_seed=8,
                       feature_fraction=0.1, 
                       feature_fraction_seed=8,
                            verbose=-1,
                       random_state=17,
                   n_jobs=-1)
                   
lgbs10 = LGBMClassifier(objective='multiclass', 
                       num_leaves=4,
                  min_data_in_leaf=400, 
                    max_depth=1, 
                     learning_rate=0.05, 
                   n_estimators=270, 
                      max_bin=60, 
                    bagging_fraction=0.5, 
                       bagging_freq=4, 
                       bagging_seed=8,
                       feature_fraction=0.15, 
                       feature_fraction_seed=8,
                            verbose=-1,
                       random_state=17,
                   n_jobs=-1)
                   
lgbs11 = LGBMClassifier(objective='multiclass', 
                       num_leaves=5,
                    min_data_in_leaf=19, 
                    max_depth=4,
                      learning_rate=0.1, 
                     n_estimators=180, 
                       max_bin=20, 
                      bagging_fraction=0.8,
                       bagging_freq=4, 
                       bagging_seed=8,
                      feature_fraction=0.1, 
                       feature_fraction_seed=8,
                          lambda_l1 = 7,
                           lambda_l2 = 0.1,
                           min_gain_to_split = 0.01,
                           metric = 'multi_logloss',
                            verbose=-1,
                       random_state=17,
                   n_jobs=-1)

'''

# Additional: Class 2 pre-modelling

Try to do modelling in sequence that first trained on separating "Class 2" from others, and then classify Class 1, 3, 4 in the remaining block. Result not satisfactory.

In [ ]:
'''
# Split features and labels
train2_labels = train_labels.apply(lambda x: 'Class_2'  if x == 'Class_2' else 'Others')
train2_features = train_features
train2_labels.unique()
'''

In [ ]:
'''
train134_labels = train.loc[train['target']!='Class_2']['target'].reset_index(drop=True)
train134_features = train.loc[train['target']!='Class_2'].drop(['id','target'], axis=1).reset_index(drop=True)
train134_labels.unique()
'''

In [ ]:
'''
# Light Gradient Boosting Regressor
lgbp2 =  LGBMClassifier(objective='binary', 
                      min_data_in_leaf=3, 
                    max_depth=4,
                      learning_rate=0.066, 
                       n_estimators=250,
                       bagging_fraction=0.75,
                       bagging_freq=4, 
                       bagging_seed=8,
                       feature_fraction=0.3,
                       feature_fraction_seed=8,
                            verbose=-1,
                       random_state=17,
                   n_jobs=-1)

#lgbp2_model = lgbp2.fit(train2_features, train2_labels)
#lgbp2_pred = lgbp2_model.predict_proba(test_features)

'''

In [ ]:
'''
# Light Gradient Boosting Regressor
lgbp134 =  LGBMClassifier(objective='multiclass', 
                      min_data_in_leaf=3, 
                    max_depth=4,
                      learning_rate=0.1, 
                       n_estimators=100,
                       bagging_fraction=0.75,
                       bagging_freq=4, 
                       bagging_seed=8,
                      feature_fraction=0.4,
                       feature_fraction_seed=8,
                            verbose=-1,
                       random_state=17,
                   n_jobs=-1)


#lgbp134_model = lgbp134.fit(train134_features, train134_labels)
#lgbp134_pred = lgbp134_model.predict_proba(test_features)
'''

In [ ]:
'''
%%time
num_est = [0.3,0.4,0.5]

vc_model = LGBMClassifier(objective='multiclass', 
                      min_data_in_leaf=3, 
                    max_depth=4,
                      learning_rate=0.1, 
                       n_estimators=100,
                       bagging_fraction=0.75,
                       bagging_freq=4, 
                       bagging_seed=8,
                      feature_fraction=0.4,
                       feature_fraction_seed=8,
                            verbose=-1,
                       random_state=17,
                   n_jobs=-1)


# Calculate accuracy on training and test set using the
# parameter with 3-fold cross validation
train_score, test_score = validation_curve( vc_model,
                                X = train134_features, y = train134_labels, 
                                param_name = 'feature_fraction', 
                                param_range = num_est, cv = 2, scoring="neg_log_loss", n_jobs=-1
                            )
 
# Calculating mean and standard deviation of training score
mean_train_score = -np.mean(train_score, axis = 1)
std_train_score = np.std(train_score, axis = 1)
 
# Calculating mean and standard deviation of testing score
mean_test_score = -np.mean(test_score, axis = 1)
std_test_score = np.std(test_score, axis = 1)
 
# Plot mean accuracy scores for training and testing scores
plt.plot(num_est, mean_train_score,
     label = "Training Score", color = 'b')
plt.plot(num_est, mean_test_score,
   label = "Cross Validation Score", color = 'g')
 
# Creating the plot
plt.title("Validation Curve")
plt.xlabel("param")
plt.ylabel("LogLoss")
plt.tight_layout()
plt.legend(loc = 'best')
plt.show()
'''

In [ ]:
#mean_test_score

In [ ]:
'''
# Read in sample_submission dataframe
submission[['Class_1']] = lgbp2_pred[:,1]*lgbp134_pred[:,0]
submission[['Class_2']] =lgbp2_pred[:,0]
submission[['Class_3']] = lgbp2_pred[:,1]*lgbp134_pred[:,1]
submission[['Class_4']] = lgbp2_pred[:,1]*lgbp134_pred[:,2]

submission.head()
'''

In [ ]:
#submission[['Class_1','Class_2','Class_3','Class_4']].sum(axis=1).unique()

In [ ]:
#submission.to_csv("submission_p2model.csv", index=False)

In [ ]:
'''
Basic lgb
public 1.09213

Tuned lgbp2>lgbp134
public 1.08981

lgbp2 = LGBMClassifier(objective='binary', 
                      min_data_in_leaf=3, 
                    max_depth=4,
                      learning_rate=0.066, 
                       n_estimators=250,
                       bagging_fraction=0.75,
                       bagging_freq=4, 
                       bagging_seed=8,
                       feature_fraction=0.3,
                       feature_fraction_seed=8,
                            verbose=-1,
                       random_state=17,
                   n_jobs=-1)
                   
                   
 lgbp134 =   LGBMClassifier(objective='multiclass', 
                      min_data_in_leaf=3, 
                    max_depth=4,
                      learning_rate=0.1, 
                       n_estimators=100,
                       bagging_fraction=0.75,
                       bagging_freq=4, 
                       bagging_seed=8,
                      feature_fraction=0.4,
                       feature_fraction_seed=8,
                            verbose=-1,
                       random_state=17,
                   n_jobs=-1)                

'''

# Additional2: Summation of features

Combinatorial summation of the 50 features as new features. The features dimension is exploding with high computation cost. Result is not satisfactory, and not good enough to justify the higher complexity.

In [ ]:
'''
trainsum_features = train_features
testsum_features = test_features
'''

In [ ]:
'''
%%time
for i in range(0,49):
    a='feature_'+str(i)
    
    for j in range(i+1,50): 
        b='feature_'+str(j)
        trainsum_features[str(i)+"_"+str(j)+"_sum"] = train_features[[a,b]].sum(axis=1)
        '''

In [ ]:
'''
%%time
for i in range(0,49):
    a='feature_'+str(i)
    
    for j in range(i+1,50): 
        b='feature_'+str(j)
        testsum_features[str(i)+"_"+str(j)+"_sum"] = test_features[[a,b]].sum(axis=1)
        '''

In [ ]:
'''
%%time
# Light Gradient Boosting Regressor
lgb =  LGBMClassifier(objective='multiclass',                     
                            verbose=-1,
                       random_state=17,
                      importance_type='gain',
                   n_jobs=-1)

lgb_model = lgb.fit(trainsum_features, train_labels)
lgb_pred = lgb_model.predict_proba(testsum_features)
'''

In [ ]:
'''
arr = np.stack((lgb.feature_name_, lgb.feature_importances_) ,axis=1)
m = [row[0] for row in arr if row[1] != "0.0"]
'''

In [ ]:
'''
trainsum_features = trainsum_features[m]
testsum_features = testsum_features[m]
'''

In [ ]:
'''
# Read in the dataset as a dataframe
trainsum_features = pd.read_csv("../input/tps-may-2021/trainsum_features.csv")
testsum_features = pd.read_csv("../input/tps-may-2021/testsum_features.csv")
'''

In [ ]:
'''
trainsum_features.to_csv("trainsum_features.csv", index=False)
testsum_features.to_csv("testsum_features.csv", index=False)
'''

In [ ]:
'''
# Light Gradient Boosting Regressor
lgbsum =  LGBMClassifier(objective='multiclass', 
                     min_data_in_leaf=1, 
                    max_depth=4,
                      learning_rate=0.07, 
                       n_estimators=100,
                       bagging_fraction=0.75,
                       bagging_freq=4, 
                       bagging_seed=8,
                     feature_fraction=0.3,
                       feature_fraction_seed=8,
                            verbose=-1,
                       random_state=17,
                   n_jobs=-1)

lgbsum_model = lgbsum.fit(trainsum_features, train_labels)
lgbsum_pred = lgbsum_model.predict_proba(testsum_features)
'''

In [ ]:
'''
# Setup cross validation folds
kf = StratifiedKFold(n_splits=5, random_state=42, shuffle=True)

# Define error metrics
def loss(y, y_pred):
    return np.sqrt(log_loss(y, y_pred))

def cvsum_loss(model, X = trainsum_features):
    loss = np.sqrt(-cross_val_score(model, X, train_labels, scoring="neg_log_loss", cv=kf, n_jobs=-1))
    return (loss)
    '''

In [ ]:
'''
scores = {}

score = cvsum_loss(lgbsum)
print("rf: {:.4f} ({:.4f})".format(score.mean(), score.std()))
scores['rf'] = (score.mean(), score.std())
'''

In [ ]:
'''
%%time
num_est = [0.05,0.07,0.1]

vc_model = LGBMClassifier(objective='multiclass', 
                     min_data_in_leaf=1, 
                    max_depth=4,
                      learning_rate=0.07, 
                       n_estimators=100,
                       bagging_fraction=0.75,
                       bagging_freq=4, 
                       bagging_seed=8,
                     feature_fraction=0.3,
                       feature_fraction_seed=8,
                            verbose=-1,
                       random_state=17,
                   n_jobs=-1)


# Calculate accuracy on training and test set using the
# parameter with 3-fold cross validation
train_score, test_score = validation_curve( vc_model,
                                X = trainsum_features, y = train_labels, 
                                param_name = 'learning_rate', 
                                param_range = num_est, cv = 2, scoring="neg_log_loss", n_jobs=-1
                            )
 
# Calculating mean and standard deviation of training score
mean_train_score = -np.mean(train_score, axis = 1)
std_train_score = np.std(train_score, axis = 1)
 
# Calculating mean and standard deviation of testing score
mean_test_score = -np.mean(test_score, axis = 1)
std_test_score = np.std(test_score, axis = 1)
 
# Plot mean accuracy scores for training and testing scores
plt.plot(num_est, mean_train_score,
     label = "Training Score", color = 'b')
plt.plot(num_est, mean_test_score,
   label = "Cross Validation Score", color = 'g')
 
# Creating the plot
plt.title("Validation Curve")
plt.xlabel("param")
plt.ylabel("LogLoss")
plt.tight_layout()
plt.legend(loc = 'best')
plt.show()
'''

In [ ]:
#mean_test_score

In [ ]:
'''
# Read in sample_submission dataframe
submission[['Class_1', 'Class_2', 'Class_3', 'Class_4']] = lgbsum_pred
submission.head()
'''

In [ ]:
#submission.to_csv("submission_sumlgb.csv", index=False)

In [ ]:
'''
Basic lgb
CV 1.0478 (0.0006)
public 1.09072

Tuned lgbsum
CV 1.0462 (0.0007)
public 1.09009

lgbsum = LGBMClassifier(objective='multiclass', 
                     min_data_in_leaf=1, 
                    max_depth=4,
                      learning_rate=0.07, 
                       n_estimators=100,
                       bagging_fraction=0.75,
                       bagging_freq=4, 
                       bagging_seed=8,
                     feature_fraction=0.3,
                       feature_fraction_seed=8,
                            verbose=-1,
                       random_state=17,
                   n_jobs=-1)
'''